# Optimizing a CUDA application

In this self-paced, hands-on lab, we will take an existing CUDA application and go through several optimization steps, measuring the performance benefits of each. We will see the importance of minimizing data transfer, enabling coalesced memory access, and tuning the parallel decomposition. 

Lab content created by EPCC, The University of Edinburgh. Documentation and source code copyright The University of Edinburgh 2016. Lab style and template created by NVIDIA, see https://nvidia.qwiklab.com/.

---
First, please try and execute the below command. Give focus to the cell by clicking on it, and then either press the play button above or press your Enter key whilst holding down Shift.

In [ ]:
!echo "This command is running on host $HOSTNAME"

## Introduction

This exercise involves performing a series of optimizations to an existing CUDA application. 

<table>
<tr>
<td>
You start with an image that looks like this:

<img src=images/input.jpeg width=400>
</td>
<td>
Which has been generated from the original:

<img src=images/EDINB00034_2048x2048.jpg width=400>


</td>

<tr>
</table>
On the left is an image of Edinburgh Castle, processed such that the edges between light and dark areas replace the original picture. Your job is to reconstruct the initial image. This is an artificial thing to do, but it mimics many scientific applications (e.g. that solve systems of PDEs) since the algorithm is iterative, requiring many successive <i>stencil</i> operations. Each pixel of the new <it>image</it> is generated based on it's neighboring pixel values and the original <it>edge</it> data by repeatedly performing the following update:  

image<sub>i,j</sub> = (image<sub>i-1,j</sub> + image<sub>i+1,j</sub> + image<sub>i,j-1</sub> + image<sub>i,j+1</sub> - edge<sub>i,j</sub>)/4 

The more iterations, the better the reconstruction (although for simplicity we work in greyscale rather
than colour).

You are provided with a working but slow CUDA implementation of the reconstruction algorithm. First of all, let’s compile and run the code. The code is set up to run the algorithm on both the GPU and the CPU. It compares the outputs from the two runs to verify correctness, and then displays timings for each run. Choose to work with either C or Fortran:


------
<b>C:</b>


In [ ]:
# set up a link to the C version of the templates
!rm -rf src; ln -s src_c src; echo "Using C version"

-------
<b>Fortran:</b>


In [ ]:
# set up a link to the Fortran version of the templates
!rm -rf src; ln -s src_fortran src; echo "Using Fortran version"

-------
Now, build and run the code:

In [ ]:
# Execute this cell to compile the reconstruction code. Wait until "Complete" is printed in the output 
!cd src; make clean; make; cd ..; echo "Complete"

In [ ]:
# Execute this cell to run the code. Wait until "Complete" is printed in the output 
!cd src; ./reconstruct; cd ..; echo "Complete" 

-----
View the resulting image:

In [ ]:
# Execute this cell to view the resulting image
!cd src; pgmtoppm white output.pgm > output.ppm; ppmtojpeg output.ppm > output.jpeg; cd ..
from IPython.core.display import Image
Image('src/output.jpeg', width=400)

Hopefully you can see that the picture is starting to become clearer. As the algorithm is iterative, there is a loop in the main routine that invokes the kernel N times where N is set to 100. Increasing N will increase the quality of the reconstruction, but please don't do this during the lab to avoid hogging the resources. If you were to run for 10 million iterations, the resulting image would look like this:

<img src=images/output10M.jpeg width=400 align="left">


Now it's time to optimise the code and improve on the GPU timing printed when you ran the code, by editing the source code. Below is a window where you can browse and edit the code. Note that a one pixel wide <it>halo</it> region of zeroes is added to each edge of the various image-data arrays; this simplifies the computation as it allows the edge pixels to be treated in the same manner as other pixels. (The edge array, which holds the original edge data, does not have require a halo.)</p>


#### Edit the source
Using the notebook (you will need a separate tab or window), navigate to the appropriate source file directory for this exercise (<code>src_c</code> or <code>src_fortran</code>) and make appropriate changes to the code. The following sections contain detailed instructions.



## Minimizing Data Transfer 

A challenge with GPUs and other accelerators is that transferring data between host memory and device memory is often relatively slow.  An important optimization technique involves minimise the amount of data that is transferred between host and device.

Notice that in the main loop in <code>reconstruct.cu</code> (C) or <code>reconstruct.cuf</code> (Fortran), the data is copied from GPU memory to host memory and then back to GPU memory at each iteration. This is not in fact necessary; with the exception of the final iteration when the data must be copied back to the host, it is going to be processed on the GPU again in the next iteration. Therefore, we can optimise manipulating the GPU memory without directly without expensive transfers.

We can simply copy the output array directly to the input array after each iteration.
In order to do this you will need to:

-----
<b>C:</b>

<ul>
<li> remove the <code>cudaMemcpy</code> calls from inside the main loop.
<li> replace them with a <code>cudaMemcpy</code> call to copy, directly on the device, from <code>d_output</code> to
  <code>d_input</code>. 
<li> add a new <code>cudaMemcpy</code> call after the end of the loop (in
  between the two calls to <code>get_current_time()</code>) to copy the
  final result back from the GPU to the <code>output</code> buffer in
  host memory. 
</ul>

-----
<b>Fortran:</b>


<ul>
<li> remove the assignments to <code>output</code> (from <code>d_output</code>) and to <code>d_input</code> (from <code>output</code>), inside the main loop.
<li> replace them with an assignment directly from <code>d_output</code> to <code>d_input</code>.
<li> add a new assignment after the end of the loop (in between the two calls to <code>cpu_time()</code>) to copy
the final result back from the GPU to the <code>output</code> buffer in host memory.
</ul>

-----
Once you have made these changes, compile and run the code again by re-executing the corresponding cells above, and take note of the time taken by the GPU version. How does it compare to the previous timing? (The server may have different types of GPUs installed, so it is possible that the GPU in use may change from run to run. For accurate timings, make sure that the same type of GPU is being used across different runs by checking the text printed at the start of the output. See the end of the "Intro" lab to see how to restrict your run to a certain GPU.)

## Enabling Coalesced Memory Accesses

The GPU performs best when consecutive CUDA threads access consecutive memory locations, allowing memory <it>coalescing</it>.

-----
<b>C:</b>

However, for the kernel in <code>reconstruct_kernels.cu</code>, it can be seen that consecutive threads correspond to consecutive rows of the image, but consecutive memory locations instead correspond to 
consecutive columns. The threads are not reading from consecutive locations.

-------

<b>Fortran:</b>

However, for the kernel in <code>reconstruct_kernels.cuf</code>, it can be seen that consecutive threads correspond to consecutive columns of the image, but consecutive memory locations instead correspond to 
consecutive rows. The threads are not reading from consecutive locations.


-------

* Update the kernel such that the role of the image row and column is reversed, in relation to how pixels are assigned to CUDA threads.
* Since the image is perfectly square, you will not need to change the way the kernel is launched. 
* How does the performance compare to the previous version?

## Improving Occupancy

You should hopefully have seen a noticeable improvement in performance as a result of the changes you made to reduce data transfers between the host and the device and to enable coalescing. However, the current solution is still sub-optimal as it will not create sufficient threads to utilise all the SMs on the GPU - it has low <it>occupancy</it>.

GPU codes typically run best when there are many threads running in parallel, each doing a small part of the work. We can achieve this with our image processing code by using a thread for each pixel of the image, rather than for each row or column as before. CUDA supports 1-, 2- or 3-dimensional decompositions. A 2D decomposition maps most naturally onto the pixels of an image.

* Update your both your kernel, and the code responsible for specifying the decomposition such that that a 2D decomposition is over both rows and columns. 
* The original code uses 256 threads per block in a 1D CUDA decomposition. Replace this with 16 threads in each of the X and Y directions of the 2D CUDA decomposition, to give a total of 256 threads per block. Ensure that the number of blocks is specified appropriately in each direction. 
* Ensure that you retain memory coalescing
* Again, measure performance and compare to the previous versions.



## Investigating Grid and Block Sizes

Once you have the 2D kernel working correctly, you can try altering certain parameters and see what effect this has on its performance. In particular, you can investigate the effects of different grid and block
sizes. How does changing the grid and block sizes affect the total runtime?

<a id="post-lab"></a>
## Keeping a copy of your work

If you wish to keep a copy of the files related to this exercise:

0. Use the notebook File menu at the top right to download the files you want, OR
1. Save this IPython Notebook by going to `File -> Download as -> IPython (.ipynb)` (or instead choose an html copy) at the top of this window.
2. You can execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.


In [ ]:
!rm -f reconstruct.zip; zip -r reconstruct.zip src; echo "Complete"

**After** executing the above cell, you should be able to download the zip file [here](/files/usertemplate/reconstruct/reconstruct.zip)

<style>
p.hint_trigger{
  margin-bottom:7px;
  margin-top:-5px;
  background:#64E84D;
}
.toggle_container{
  margin-bottom:0px;
}
.toggle_container p{
  margin:2px;
}
.toggle_container{
  background:#f0f0f0;
  clear: both;
  font-size:100%;
}
</style>
<script>
$("p.hint_trigger").click(function(){
   $(this).toggleClass("active").next().slideToggle("normal");
});
   
$(".toggle_container").hide();
</script>